# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

Carregamento das Bibliotecas e Dados

In [9]:
import pandas as pd
from tkinter import Tk
from tkinter.filedialog import askopenfilename

# Função para selecionar e carregar um arquivo .csv
def selecionar_arquivo_csv():
    Tk().withdraw()  # Esconde a janela principal do Tkinter
    caminho_arquivo = askopenfilename(
        title="Selecione o arquivo CSV",
        filetypes=[("Arquivos CSV", "*.csv"), ("Todos os arquivos", "*.*")]
    )
    
    if caminho_arquivo:
        try:
            # Carregar os dados do arquivo .csv
            df = pd.read_csv(caminho_arquivo)
            print("\nArquivo CSV carregado com sucesso!")
            print(df.info())  # Exibe informações do DataFrame
            print(df.head())  # Exibe as primeiras linhas do DataFrame
            return df
        except Exception as e:
            print(f"Erro ao carregar o arquivo CSV: {e}")
            return None
    else:
        print("Nenhum arquivo CSV foi selecionado.")
        return None

# Chamar a função para selecionar e carregar o arquivo
df = selecionar_arquivo_csv()

# Verificar se o DataFrame foi carregado com sucesso
if df is not None:
    print("Dados carregados com sucesso!")
else:
    print("O carregamento do arquivo falhou ou foi cancelado.")



Arquivo CSV carregado com sucesso!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda             

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

Tratamento de Dados

In [11]:
# Preencher valores nulos na coluna 'tempo_emprego' com a mediana
df['tempo_emprego'] = df['tempo_emprego'].fillna(df['tempo_emprego'].median())

# Checar valores nulos restantes
print(df.isnull().sum())


Unnamed: 0               0
data_ref                 0
id_cliente               0
sexo                     0
posse_de_veiculo         0
posse_de_imovel          0
qtd_filhos               0
tipo_renda               0
educacao                 0
estado_civil             0
tipo_residencia          0
idade                    0
tempo_emprego            0
qt_pessoas_residencia    0
renda                    0
dtype: int64


Criação do Modelo

In [20]:
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm

# Carregar os dados
df = pd.read_csv('C:\\Users\\conta\\OneDrive\\Área de Trabalho\\Nova pasta\\previsao_de_renda.csv')

# Preencher valores nulos na coluna 'tempo_emprego' com a mediana
df['tempo_emprego'] = df['tempo_emprego'].fillna(df['tempo_emprego'].median())

# Função para ajustar o modelo e exibir o resumo
def ajustar_modelo(df, formula):
    y, X = dmatrices(formula, data=df, return_type='dataframe')
    model = sm.OLS(y, X).fit()
    return model

# Fórmula inicial
formula_inicial = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia'

# Ajustar o modelo inicial
modelo_inicial = ajustar_modelo(df, formula_inicial)
print("Resumo do Modelo Inicial:")
print(modelo_inicial.summary())

# Iterar removendo variáveis menos significativas (p-value > 0.05)
# Com base no resumo do modelo inicial, você pode identificar quais variáveis remover

# Exemplo de fórmula ajustada após remoção de variáveis
formula_ajustada = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + estado_civil + qtd_filhos + idade + tempo_emprego'

# Ajustar o modelo ajustado
modelo_ajustado = ajustar_modelo(df, formula_ajustada)
print("\nResumo do Modelo Ajustado:")
print(modelo_ajustado.summary())

# Comparar os modelos
print("\nComparação dos Modelos:")
print(f"R-quadrado do Modelo Inicial: {modelo_inicial.rsquared:.4f}")
print(f"R-quadrado do Modelo Ajustado: {modelo_ajustado.rsquared:.4f}")
print(f"R-quadrado Ajustado do Modelo Inicial: {modelo_inicial.rsquared_adj:.4f}")
print(f"R-quadrado Ajustado do Modelo Ajustado: {modelo_ajustado.rsquared_adj:.4f}")

# Observações:
# - Analise o R-quadrado e R-quadrado ajustado para verificar se houve melhoria.
# - Examine o resumo dos modelos para avaliar as variáveis significativas e p-values.


Resumo do Modelo Inicial:
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.348
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     333.6
Date:                Sat, 17 Aug 2024   Prob (F-statistic):               0.00
Time:                        00:11:04   Log-Likelihood:                -16213.
No. Observations:               15000   AIC:                         3.248e+04
Df Residuals:                   14975   BIC:                         3.267e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

Iteração para Melhorar o Modelo

In [3]:
import pandas as pd
import statsmodels.api as sm
from patsy import dmatrices
import numpy as np  # Importação do numpy

# Função para ajustar o modelo e exibir o resumo
def ajustar_modelo(df, formula):
    y, X = dmatrices(formula, data=df, return_type='dataframe')
    model = sm.OLS(y, X).fit()
    return model

# Fórmula do modelo
formula_inicial = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia + qtd_filhos + idade + tempo_emprego + qt_pessoas_residencia'

# Ajustar o modelo inicial
modelo_inicial = ajustar_modelo(df, formula_inicial)
print("Resumo do Modelo Inicial:")
print(modelo_inicial.summary())


Resumo do Modelo Inicial:
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     165.3
Date:                Tue, 07 Jan 2025   Prob (F-statistic):               0.00
Time:                        15:41:41   Log-Likelihood:                -13204.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12441   BIC:                         2.664e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

4. Criando o Segundo Modelo

In [5]:
# Fórmula do modelo com variáveis removidas
formula_segundo_modelo = 'np.log(renda) ~ sexo + posse_de_imovel + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego'

# Ajustar o segundo modelo
modelo_segundo = ajustar_modelo(df, formula_segundo_modelo)
print("Resumo do Segundo Modelo:")
print(modelo_segundo.summary())


Resumo do Segundo Modelo:
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     188.7
Date:                Tue, 07 Jan 2025   Prob (F-statistic):               0.00
Time:                        15:46:14   Log-Likelihood:                -13206.
No. Observations:               12466   AIC:                         2.646e+04
Df Residuals:                   12444   BIC:                         2.662e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------

6. O Código do Terceiro Modelo Ficaria Assim:

In [7]:
# Fórmula do modelo com mais variáveis removidas
formula_terceiro_modelo = 'np.log(renda) ~ sexo + posse_de_imovel + tipo_renda + educacao + tipo_residencia + idade + tempo_emprego'

# Ajustar o terceiro modelo
modelo_terceiro = ajustar_modelo(df, formula_terceiro_modelo)
print("Resumo do Terceiro Modelo:")
print(modelo_terceiro.summary())


Resumo do Terceiro Modelo:
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.241
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     232.8
Date:                Tue, 07 Jan 2025   Prob (F-statistic):               0.00
Time:                        15:47:05   Log-Likelihood:                -13209.
No. Observations:               12466   AIC:                         2.645e+04
Df Residuals:                   12448   BIC:                         2.659e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------

7. Explicação Final e Conclusão do Projeto

Objetivo do Projeto
O objetivo deste projeto foi ajustar um modelo de regressão linear múltipla para entender os determinantes da variável np.log(renda) (logaritmo da renda) a partir de uma série de variáveis explicativas, como características demográficas, educacionais e socioeconômicas. A análise foi conduzida em três etapas principais: o modelo inicial, a remoção de variáveis insignificantes para o segundo modelo, e uma análise adicional para a criação de um terceiro modelo, removendo ainda mais variáveis com baixo poder explicativo.

1. Modelo Inicial
O modelo inicial foi ajustado com uma série de variáveis explicativas e apresentou um R² ajustado de 0.240. Esse valor indica que cerca de 24% da variabilidade da renda é explicada pelas variáveis do modelo. Os resultados do modelo inicial foram revisados para avaliar a significância das variáveis, com base nos p-valores e coeficientes.

2. Remoção das Variáveis Insignificantes
A partir da análise do summary do modelo, identifiquei algumas variáveis com p-valores muito altos (acima de 0.05), indicando que elas não contribuem de forma significativa para o modelo. As variáveis removidas foram:

posse_de_veiculo[T.True] (p-valor = 0.931),
tipo_renda[T.Bolsista] (p-valor = 0.888),
estado_civil[T.Separado] (p-valor = 0.738).
A justificativa para a remoção dessas variáveis foi que seus p-valores altos indicam que elas não têm uma associação significativa com a renda. A remoção dessas variáveis permite simplificar o modelo sem perder poder explicativo substancial, mantendo apenas variáveis com forte relevância estatística.

3. Segundo Modelo
Com as variáveis menos significativas removidas, ajustei o segundo modelo, que apresentou um ajuste mais simples e ainda explicativo, sem comprometer a capacidade de explicar a variável dependente. O R² ajustado foi mantido em níveis satisfatórios, e os coeficientes das variáveis restantes passaram a ser mais relevantes, com p-valores menores e mais consistentes.

4. Análise Adicional para o Terceiro Modelo
O terceiro modelo foi desenvolvido aplicando o mesmo critério de remoção das variáveis insignificantes, focando em variáveis que ainda apresentavam p-valores superiores a 0.05. Variáveis como tipo_renda[T.Pensionista] (p-valor = 0.105) e tipo_residencia[T.Com os pais] (p-valor = 0.339) foram removidas devido à baixa contribuição explicativa.

O objetivo dessa etapa foi refinar ainda mais o modelo e verificar se a remoção de outras variáveis com p-valor mais alto poderia melhorar o ajuste sem perder a qualidade do modelo.

5. Comparação dos Modelos
Durante o processo de criação e refinamento dos modelos, fizemos a comparação entre o AIC (Akaike Information Criterion), BIC (Bayesian Information Criterion) e R² ajustado dos diferentes modelos. Os critérios AIC e BIC são importantes para avaliar a qualidade do modelo, levando em consideração a quantidade de variáveis explicativas e o poder explicativo. Quanto menor o AIC e o BIC, melhor o modelo em termos de parsimônia (simples e eficaz).

Os valores de R² ajustado indicaram que os modelos posteriores não apresentaram uma perda significativa de poder explicativo, ao mesmo tempo em que reduziram a complexidade do modelo, removendo variáveis desnecessárias. Em termos gerais, o segundo e terceiro modelos mostraram uma boa combinação entre simplicidade e precisão explicativa.

Conclusão
Ao longo do processo, foi possível ajustar o modelo de forma eficiente, removendo variáveis que não contribuíam significativamente para a explicação da renda. As principais decisões e etapas foram:

Remoção das variáveis insignificantes com base nos p-valores.
Simplificação do modelo, mantendo apenas as variáveis mais significativas.
Avaliação do impacto de cada variável na explicação do modelo, utilizando métricas como AIC, BIC e R² ajustado.
O modelo final tem como resultado um conjunto de variáveis explicativas que são significativamente associadas à renda, como sexo, posse de imóvel, tipo de renda e idade. Esse processo de refinamento do modelo não apenas melhora a explicação da variável dependente, mas também torna a análise mais clara e objetiva.